In [ ]:
#default_exp radar.v1
from nbdev.showdoc import show_doc

# radar.v1

RadarFrame encapsulates the signal processing for getting different stages of the standard FMCW radar processing pipeline.

More details can be found at [V1RadarExamples]

In [ ]:
#hide

#For an easier life when developing
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

import numpy as np
from mmwave import dsp

In [ ]:
#| export

class RadarFrame(object):
    """Encapsulates processing of a radar frame data cube.
    The RadarFrame object can be initialized once, so long as there's no change in
    the radar configuration.
    When new radar data is available, set raw radar data using the `raw_cube` property.

    Various processing of the radar data are computed lazily on the first
    access to each property. Processing can also be triggered manually by calling
    the related methods.

    This object stores intemediate steps to avoid recomputation when possible.
    If no new raw datacube is passed in, subsequent request for different views of
    the radar data are returned from the stored state.
    """
    def __init__(self, radar_config,
                 angle_res = 1,
                 angle_range = 90,
                 origin_at_bottom_center = True,
                 use_float32 = False,
                ):
        super(RadarFrame, self).__init__()
        self.cfg = radar_config

        #Beamforming params
        self.bins_processed = self.cfg['profiles'][0]['adcSamples'] #radar_cube.shape[0]
        self.virt_ant = self.cfg['numLanes'] * len(self.cfg['chirps']) #radar_cube.shape[1]
        self.__doppler_bins = self.cfg['numChirps'] // len(self.cfg['chirps']) #radar_cube.shape[2]
        self.angle_res = angle_res
        self.angle_range = angle_range
        self.angle_bins = (self.angle_range * 2) // self.angle_res + 1
        self.num_vec, self.steering_vec = dsp.gen_steering_vec(self.angle_range,
                                                               self.angle_res,
                                                               self.virt_ant)

        #Properties
        self.__range_azimuth_dirty = True
        self.__range_azimuth = np.zeros((self.bins_processed, self.angle_bins),
                                        dtype=np.complex64 if use_float32 else np.complex_)
        self.__beam_weights = np.zeros((self.virt_ant, self.bins_processed),
                                       dtype=np.complex64 if use_float32 else np.complex_)
        self.__range_doppler = None
        self.__raw_cube = None
        self.__range_cube = None

        self.__flip_ra = origin_at_bottom_center

    @property
    def flipped(self):
        """True if zero range is at bottom center of image,
        i.e. y is flipped from typical image pixel coordinates with 0,0 at top left corner.
        This is important for computing the intrinsic matrix"""
        return self.__flip_ra

    @property
    def range_nbins(self):
        """Number of bins in range"""
        return self.bins_processed

    @property
    def max_range(self):
        """Maximum range in meters, this property is computed from the radar configuration"""
        return self.range_resolution * self.range_nbins

    @property
    def range_resolution(self):
        """Range resolution in meters"""
        range_res, bw = dsp.range_resolution(self.cfg['profiles'][0]['adcSamples'],
                                             self.cfg['profiles'][0]['adcSampleRate'] / 1000,
                                             self.cfg['profiles'][0]['freqSlopeConst'] / 1e12)
        return range_res
    
    @property
    def doppler_bins(self):
        """Number of bins in doppler dimension"""
        return self.__doppler_bins

    @property
    def doppler_resolution(self):
        """Doppler resolution in m/s"""
        _, bw = dsp.range_resolution(self.cfg['profiles'][0]['adcSamples'],
                                             self.cfg['profiles'][0]['adcSampleRate'] / 1000,
                                             self.cfg['profiles'][0]['freqSlopeConst'] / 1e12)
        return dsp.doppler_resolution(bw,
                                      start_freq_const=self.cfg['profiles'][0]['start_frequency'] / 1e9,
                                      ramp_end_time=self.cfg['profiles'][0]['rampEndTime'] * 1e6,
                                      idle_time_const=self.cfg['profiles'][0]['idle'] * 1e6,
                                      num_loops_per_frame=self.cfg['numChirps'] / len(self.cfg['chirps']),
                                      num_tx_antennas=self.cfg['numTx'])


    @property
    def max_unambiguous_doppler(self):
        """Maximum unambiguous doppler in m/s.
        In FMCW radar processing, aliasing will occur when the object's relative Doppler
        is above this value.
        i.e. if `max_unambiguous_doppler` is 10m/s, and the object is at 11 m/s, the computed doppler will be -9 m/s.
        """
        return self.doppler_resolution * self.doppler_bins / 2
        raise NotImplementedError

    @property
    def raw_cube(self):
        """Raw radar data cube
        Set the radar data cube using this property.
        `rf.raw_cube = radar_cube`

        This will clear all cached processed data like `range_cube`, `range_azimuth`, etc.
        """
        return self.__raw_cube

    @raw_cube.setter
    def raw_cube(self, raw_cube):
        self.__raw_cube = raw_cube
        self.__range_cube = None
        self.__range_doppler = None
        self.__range_azimuth_dirty = True

    @property
    def range_cube(self):
        """Get the radar data as a range cube.
        Processing from the raw cube is done lazily on the first access to this property.
        """
        if self.__range_cube is not None:
            return self.__range_cube
        else:
            range_cube = dsp.range_processing(self.raw_cube)
            self.__range_cube = np.swapaxes(range_cube, 0, 2)
            return self.__range_cube

    @property
    def range_doppler(self):
        """Get the radar data as a range doppler cube.
        Processing from the raw cube is done lazily on the first access to this property.
        """
        if self.__range_doppler is not None:
            return self.__range_doppler
        else:
            range_doppler = dsp.doppler_processing(self.raw_cube)
            self.__range_doppler = range_doppler
            return self.__range_doppler

    @property
    def range_azimuth_capon(self):
        """Get the radar data as a range azimuth cube,
        beamformed using the Capon beamformer.
        This property is computed lazily on first access.
        """
        if not self.__range_azimuth_dirty:
            r = self.__range_azimuth
        else:
            self.__aoa_capon_process()
            r = self.__range_azimuth

        if self.__flip_ra:
            return np.flipud(np.fliplr(r))
        else:
            return r

    def __aoa_capon_process(self):
        radar_cube = self.range_cube

        for jj in range(self.bins_processed):
            self.__range_azimuth[jj,:], self.__beam_weights[:,jj] = dsp.aoa_capon(radar_cube[jj],
                                                                      self.steering_vec)

        self.__range_azimuth_dirty = False

    def compute_range_azimuth(self, radar_raw=None, method='capon'):
        """Beamform raw radar datacube
        Use this method to set the the raw radar data cube and 
        perform beamforming. The beamformed data cube will be returned.
        
        Currently only the capon method is implemented.

        This is a convenience method that is equivalent to:
        ```
        # rf = RadarFrame(...)
        rf.raw_cube = radar_raw
        beamformed_datacube = rf.range_azimuth_capon
        ```
        """
        if radar_raw is not None:
            self.raw_cube = radar_raw

        if method == 'capon':
            return self.range_azimuth_capon
        else:
            raise NotImplementedError

In [ ]:
show_doc(RadarFrame.range_nbins)
show_doc(RadarFrame.max_range)
show_doc(RadarFrame.range_resolution)
show_doc(RadarFrame.doppler_resolution)
show_doc(RadarFrame.max_unambiguous_doppler)
show_doc(RadarFrame.doppler_bins)
show_doc(RadarFrame.raw_cube)
show_doc(RadarFrame.range_cube)
show_doc(RadarFrame.range_doppler)
show_doc(RadarFrame.range_azimuth_capon)
show_doc(RadarFrame.compute_range_azimuth)

<h4 id="RadarFrame.range_nbins" class="doc_header"><code>RadarFrame.range_nbins</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Number of bins in range

<h4 id="RadarFrame.max_range" class="doc_header"><code>RadarFrame.max_range</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Maximum range in meters, this property is computed from the radar configuration

<h4 id="RadarFrame.range_resolution" class="doc_header"><code>RadarFrame.range_resolution</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Range resolution in meters

<h4 id="RadarFrame.doppler_resolution" class="doc_header"><code>RadarFrame.doppler_resolution</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Doppler resolution in m/s

<h4 id="RadarFrame.max_unambiguous_doppler" class="doc_header"><code>RadarFrame.max_unambiguous_doppler</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Maximum unambiguous doppler in m/s.
In FMCW radar processing, aliasing will occur when the object's relative Doppler
is above this value.
i.e. if `max_unambiguous_doppler` is 10m/s, and the object is at 11 m/s, the computed doppler will be -9 m/s.

...

<h4 id="RadarFrame.raw_cube" class="doc_header"><code>RadarFrame.raw_cube</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Raw radar data cube
Set the radar data cube using this property.
`rf.raw_cube = radar_cube`

This will clear all cached processed data like `range_cube`, `range_azimuth`, etc.

<h4 id="RadarFrame.range_cube" class="doc_header"><code>RadarFrame.range_cube</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Get the radar data as a range cube.
Processing from the raw cube is done lazily on the first access to this property.

<h4 id="RadarFrame.range_doppler" class="doc_header"><code>RadarFrame.range_doppler</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Get the radar data as a range doppler cube.
Processing from the raw cube is done lazily on the first access to this property.

<h4 id="RadarFrame.range_azimuth_capon" class="doc_header"><code>RadarFrame.range_azimuth_capon</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Get the radar data as a range azimuth cube,
beamformed using the Capon beamformer.
This property is computed lazily on first access.

<h4 id="RadarFrame.compute_range_azimuth" class="doc_header"><code>RadarFrame.compute_range_azimuth</code><a href="__main__.py#L170" class="source_link" style="float:right">[source]</a></h4>

> <code>RadarFrame.compute_range_azimuth</code>(**`radar_raw`**=*`None`*, **`method`**=*`'capon'`*)

Beamform raw radar datacube
Use this method to set the the raw radar data cube and 
perform beamforming. The beamformed data cube will be returned.

Currently only the capon method is implemented.

This is a convenience method that is equivalent to:
```
# rf = RadarFrame(...)
rf.raw_cube = radar_raw
beamformed_datacube = rf.range_azimuth_capon
```